In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc

from transformers import AutoTokenizer, PreTrainedTokenizerFast
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, normalizers
from konlpy.tag import Okt, Kkma, Mecab
import sentencepiece as spm

import os
import re
import math
import json
import random
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/user/anaconda3/envs/torch_gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### config 정의

In [2]:
label_dict = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
class_names = ['협박 대화', '갈취 대화', '직장괴롭힘 대화', '기타괴롭힘 대화', '일반 대화']
class_map = {0: '협박 대화', 1: '갈취 대화', 2: '직장 내 괴롭힘 대화', 3: '기타 괴롭힘 대화', 4: '일반 대화'}

In [3]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [4]:
def font_setting():
    plt.rc('font', family='NanumGothic')
    plt.rcParams['axes.unicode_minus'] = False
    sns.set_theme(style="whitegrid", font='NanumGothic', palette="muted")

font_setting()

In [5]:
def init_weights(module):
    """BERT 표준 가중치 초기화 (std=0.01)"""
    if isinstance(module, (nn.Linear, nn.Embedding)):
        module.weight.data.normal_(mean=0.0, std=0.01)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()
    elif isinstance(module, nn.LayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)

In [ ]:
class BertConfig:
    def __init__(self):
        # 모델 구조
        self.vocab_size = 35000
        self.d_model = 256
        self.num_layers = 4
        self.num_heads = 8
        self.ff_dim = 1024
        self.max_pos = 128  # 문장의 최대 길이
        self.dropout = 0.3
        
        #모델 경로
        self.model_path = None

        # 학습 설정
        self.max_seq_len = 128 # 토큰 개수 기준
        self.batch_size = 16
        self.learning_rate = 5e-5
        self.weight_decay = 0.1
        self.epochs = 100
        self.patience = 20
        
        # 특수 토큰 ID
        self.force_train = True             # "wordpiece", "spm" 재학습 시
        self.tokenizer_type = 'pretrained'   # "wordpiece", "pretrained", "spm", 'okt', 'kkma', 'mecab'
        self.spm_model_type = 'unigram'      # 'bpe', 'unigram'
        self.pretrained_model_name = "klue/bert-base"  # 한국어 전용 사전학습 모델 예시
        self.pad_id = 0
        self.unk_id = 1
        self.cls_id = 2
        self.sep_id = 3
        self.mask_id = 4

        self.pad_token = '[PAD]'
        self.unk_token = '[UNK]'
        self.cls_token = '[CLS]'
        self.sep_token = '[SEP]'
        self.mask_token = '[MASK]'

    def update_from_tokenizer(self, tokenizer):
        """
        최종적으로 토크나이저의 상태를 config에 동기화합니다.
        """
        # 1. 실제 사용 중인 토크나이저의 어휘 사전 크기로 업데이트
        self.vocab_size = len(tokenizer)

        # 2. [보완] SPTokenizer인 경우, 실제 할당된 mask_id를 가져오는 로직 추가
        # 만약 tokenizer에 word2idx가 있다면 직접 조회하는 것이 가장 정확합니다.
        if hasattr(tokenizer, 'word2idx'):
            self.mask_id = tokenizer.word2idx.get('[MASK]', self.mask_id)

        # 3. ID 중복 여부 체크 (더 포괄적으로 변경)
        ids = [self.pad_id, self.unk_id, self.cls_id, self.sep_id]
        if len(ids) != len(set(ids)):
            print(f">> [Warning] 특수 토큰 ID 간에 중복이 감지되었습니다! 현재 상태: {ids}")
            # 필요 시 여기서 강제 재조정 로직 수행

        # 4. 최종 확정된 ID 상태 출력
        print(f"[Config Finalized] Vocab: {self.vocab_size}")
        # mask_token 대신 mask_id를 출력하여 숫자값을 확인하세요.
        print(f" >> IDs - PAD:{self.pad_id}, UNK:{self.unk_id}, CLS:{self.cls_id}, SEP:{self.sep_id}, MASK:{self.mask_id}")
    
    def save(self, path):
        with open(path, 'w') as f:
            json.dump(self.__dict__, f, indent=4)

    @staticmethod
    def load(path):
        with open(path, 'r') as f:
            data = json.load(f)
        config = BertConfig()
        config.__dict__.update(data)
        return config
    
config = BertConfig()    

---
### utils

In [7]:
def model_tagging(config):
    if config.tokenizer_type == "pretrained":
        model_tag = config.pretrained_model_name.replace("/", "_")
    elif config.tokenizer_type == "spm":
        model_tag = f"spm_{config.spm_model_type}"
    elif config.tokenizer_type == "wordpiece":
        model_tag = "wordpiece" # 혹은 "wp"
    else:
        model_tag = config.tokenizer_type
    return model_tag

In [8]:
def save_training_plots(history, model_tag, all_labels, logits, num_labels=5):
    """
    학습 결과(Loss/Acc)와 클래스별 ROC 커브를 시각화하여 저장합니다.
    """
    font_setting()
    # 0. 클래스 맵 설정 (범례 표시용)
    class_names = ['Threat', 'Extortion', 'Workplace', 'Other', 'Normal']
    save_dir = f"./results/{model_tag}"
    os.makedirs(save_dir, exist_ok=True)
    
    # 1. Train vs Val (Loss & Acc) 그리기
    epochs = range(1, len(history['train_loss']) + 1)
    plt.style.use('seaborn-v0_8-muted') # 깔끔한 테마 적용
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Loss Plot
    ax1.plot(epochs, history['train_loss'], 'b-o', label='Train Loss', markersize=4)
    ax1.plot(epochs, history['val_loss'], 'r-o', label='Val Loss', markersize=4)
    ax1.set_title('Learning Curve: Loss', fontsize=13, fontweight='bold')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.legend()

    # Accuracy Plot
    ax2.plot(epochs, history['train_acc'], 'b-o', label='Train Acc', markersize=4)
    ax2.plot(epochs, history['val_acc'], 'r-o', label='Val Acc', markersize=4)
    ax2.set_title('Learning Curve: Accuracy', fontsize=13, fontweight='bold')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.grid(True, linestyle='--', alpha=0.6)
    ax2.legend()
    
    plt.tight_layout()
    plt.savefig(f"{save_dir}/learning_curves.png", dpi=300)
    plt.close()

    # 2. ROC Curve 계산 및 그리기 (Multi-class OvR)
    # [수정] logits가 리스트 형태의 텐서들일 경우를 대비해 안전하게 병합
    all_logits_tensor = torch.tensor(np.array(logits)).float()

    # 마지막 차원(dim=-1)을 기준으로 소프트맥스 적용
    y_score = torch.softmax(all_logits_tensor, dim=-1).numpy()
    
    # 정답 레이블 이진화 (0 -> [1,0,0,0,0])
    y_test = label_binarize(all_labels, classes=list(range(num_labels)))
    
    plt.figure(figsize=(9, 7))
    
    # 각 클래스별로 ROC 계산
    for i in range(num_labels):
        fpr, tpr, _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc = auc(fpr, tpr)
        
        # 실제 클래스 이름을 사용하여 그래프 그림
        label_name = class_names[i] if i < len(class_names) else f'Class {i}'
        plt.plot(fpr, tpr, lw=2, label=f'{label_name} (AUC = {roc_auc:.3f})')
        
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=1) # 기준선
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.title(f'Multi-class ROC Curve ({model_tag})', fontsize=14, fontweight='bold')
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(alpha=0.3)
    
    plt.savefig(f"{save_dir}/roc_curve.png", dpi=300)
    plt.close()
    
    print(f">> [Success] 시각화 결과가 저장되었습니다: {save_dir}")


In [9]:
def plot_and_save_confusion_matrix(y_true, y_pred, class_names, save_path):
    font_setting()
    # 1. 혼동 행렬 계산
    cm = confusion_matrix(y_true, y_pred)

    # 2. 그래프 설정
    plt.figure(figsize=(10, 8)) # 그림 크기 설정
    
    # Seaborn 히트맵 그리기
    # annot=True: 셀 안에 숫자 표시, fmt='d': 정수형 포맷, cmap: 색상 테마
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    
    # 축 라벨 및 제목 설정
    plt.xlabel('Predicted Label') # 모델이 예측한 라벨
    plt.ylabel('True Label')      # 실제 정답 라벨
    plt.title('Confusion Matrix (Validation Set)')
    plt.xticks(rotation=45) # X축 라벨이 길 경우를 대비해 회전

    # 3. 이미지 저장 및 닫기
    plt.tight_layout() # 여백 자동 조정
    plt.savefig(save_path, dpi=300) # 고해상도 저장
    plt.close() # 메모리 해제를 위해 플롯 닫기
    print(f">> Confusion Matrix image saved to {save_path}")

In [10]:
def debug(train_loader, tokenizer):
     # 1. DataLoader에서 배치 하나 가져오기
    sample_batch = next(iter(train_loader))
    ids, labels = sample_batch

    # 2. 첫 번째 데이터 샘플링
    first_sample = ids[0].tolist()

    # 3. 육안 검증 (숫자 확인)
    print("--- [1] 숫자 ID 검증 ---")
    print(f"맨 앞 (CLS 예상): {first_sample[0]} (2이면 성공)")
    # 0(PAD)이 나오기 직전 값이 3(SEP)인지 확인
    last_token_idx = (ids[0] != 0).sum().item() - 1 
    print(f"문장 끝 (SEP 예상): {first_sample[last_token_idx]} (3이면 성공)")
    print(f"패딩 시작 (PAD 예상): {first_sample[last_token_idx + 1] if last_token_idx + 1 < 128 else 'N/A'} (0이면 성공)")

    # 4. 문자열 검증 (토크나이저 복원)
    print("\n--- [2] 토크나이저 디코드 검증 ---")
    decoded = tokenizer.decode(first_sample)
    print(f"복원된 문장: {decoded}")

In [11]:
def save_checkpoint(model, optimizer, config, epoch, f1, loss, path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'config': config.__dict__,
        'val_f1': f1, 
        'val_loss': loss
    }, path)


---
### 모델 선언

In [12]:
# 4. 추론용 데이터셋
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, config):
        self.texts = texts
        self.tokenizer = tokenizer
        self.config = config

    def __len__(self): return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        tokens = self.tokenizer.encode(text)
        encoded = [self.config.cls_id] + tokens + [self.config.sep_id]
        
        if len(encoded) < self.config.max_seq_len:
            encoded += [self.config.pad_id] * (self.config.max_seq_len - len(encoded))
        else:
            encoded = encoded[:self.config.max_seq_len]
        return torch.tensor(encoded)

In [ ]:
class StandardBertModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        # 1. 임베딩 층 (Token + Position + Segment)
        self.token_emb = nn.Embedding(config.vocab_size, config.d_model)
        self.pos_emb = nn.Embedding(config.max_pos, config.d_model)
        self.seg_emb = nn.Embedding(2, config.d_model)
        
        self.norm = nn.LayerNorm(config.d_model, eps=1e-12)
        self.dropout = nn.Dropout(config.dropout)
        
        # 2. 트랜스포머 인코더 블록들
        self.layers = nn.ModuleList([
            nn.ModuleDict({
                # MHA: Scaled Dot-Product Attention 기반 병렬 헤드
                'mha': nn.MultiheadAttention(config.d_model, config.num_heads, 
                                             dropout=config.dropout, batch_first=True),
                'norm1': nn.LayerNorm(config.d_model, eps=1e-12),
                # FFN: Position-wise Feed Forward Network
                'ffn': nn.Sequential(
                    nn.Linear(config.d_model, config.ff_dim),
                    nn.GELU(),
                    nn.Dropout(config.dropout),
                    nn.Linear(config.ff_dim, config.d_model),
                    nn.Dropout(config.dropout)
                ),
                'norm2': nn.LayerNorm(config.d_model, eps=1e-12)
            }) for _ in range(config.num_layers)
        ])
        
        # 3. Pooler: 토큰의 출력을 압축하여 문장 벡터 생성
        self.pooler = nn.Sequential(
            nn.Linear(config.d_model, config.d_model),
            nn.Tanh()
        )

    def forward(self, input_ids, token_type_ids=None, mask=None):
        batch_size, seq_len = input_ids.size()
        
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        
        # 위치 ID 생성 및 배치 크기에 맞게 확장
        pos_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0)
        pos_ids = pos_ids.expand(batch_size, seq_len)
        
        # 임베딩 합산: 핵심 문맥 정보 결합
        x = self.token_emb(input_ids) + self.pos_emb(pos_ids) + self.seg_emb(token_type_ids)
        x = self.norm(x)
        x = self.dropout(x)

        # MHA용 패딩 마스크 생성 (0인 곳을 True로 변환하여 연산 제외)
        padding_mask = (mask == 0) if mask is not None else None

        # 인코더 레이어 순회
        for layer in self.layers:
            # 1) Multi-Head Attention + Residual + LayerNorm
            attn_out, _ = layer['mha'](x, x, x, key_padding_mask=padding_mask)
            x = layer['norm1'](x + attn_out)
            
            # 2) Feed Forward Network + Residual + LayerNorm
            ffn_out = layer['ffn'](x)
            x = layer['norm2'](x + ffn_out)
            
        # 전체 시퀀스 출력 x와 [CLS] 토큰 기반의 문장 벡터 반환
        return x, self.pooler(x[:, 0])

In [14]:
class BertForSequenceClassification(nn.Module):
    def __init__(self, backbone, config, num_labels):
        super().__init__()
        self.bert = backbone
        self.dropout = nn.Dropout(config.dropout)
        # 최종 분류기: 문장 벡터를 입력받아 클래스 개수만큼 출력
        self.classifier = nn.Linear(config.d_model, num_labels)

    def forward(self, input_ids, token_type_ids=None, mask=None):
        # backbone에서 pooler 출력을 가져옴
        _, pooled_output = self.bert(input_ids, token_type_ids, mask)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

---
### Data Management Module

In [15]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'[^가-힣]', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()   

In [ ]:
class ClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, config, is_train=False, mask_prob=0.25):
        self.tokenizer = tokenizer
        self.config = config
        self.labels = labels
        self.is_train = is_train  # 훈련 데이터셋일 때만 마스킹 적용
        self.mask_prob = mask_prob # 마스킹할 확률 (기본 25%)
        self.data = []

        for t in texts:
            tokens = self.tokenizer.encode(t)
            tokens = tokens[:config.max_seq_len - 2]
            formatted_tokens = [config.cls_id] + tokens + [config.sep_id]
            self.data.append(torch.tensor(formatted_tokens, dtype=torch.long))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
            tokens = self.data[idx].clone()
            label = torch.tensor(self.labels[idx], dtype=torch.long)
            
            # target_len은 조건문 상관없이 항상 필요하므로 맨 위나 밖에서 정의합니다.
            target_len = self.config.max_seq_len 

            # 2. 훈련 시에만 Input Masking 적용
            if self.is_train:
                actual_len = len(tokens)
                # CLS(0) ~ SEP(actual_len-1) 사이만 마스킹
                for i in range(1, actual_len - 1): 
                    
                    # 25% 확률로 마스킹 대상 선정
                    if random.random() < self.mask_prob: 
                        # 80-10-10 룰 적용
                        dice = random.random()
                        
                        if dice < 0.8:
                            # 80%: [MASK] 토큰으로 교체
                            tokens[i] = getattr(self.config, 'mask_id', 4)
                        elif dice < 0.9:
                            # 10%: 랜덤 단어로 교체
                            tokens[i] = random.randint(0,  self.config.vocab_size - 1)
                        else:
                            # 10%: 원래 단어 유지
                            pass
            
            # 너무 길면 자르기 (Truncation)
            if len(tokens) > target_len:
                tokens = tokens[:target_len]
                tokens[-1] = self.config.sep_id 

            # 너무 짧으면 늘리기 (Padding)
            elif len(tokens) < target_len:
                pad_len = target_len - len(tokens)
                pad_tensor = torch.full((pad_len,), self.config.pad_id, dtype=torch.long)
                tokens = torch.cat([tokens, pad_tensor])
            
            return tokens, label

In [17]:
def balance_dataset(df, class_column='class'):
    """명시적 반복문을 사용하여 데이터를 클래스별로 균등하게 샘플링합니다."""
    # 1. 각 클래스별 최소 개수 확인
    min_count = df[class_column].value_counts().min()
    print(f">> [Data Balancing] 최소 데이터 클래스 개수 기준: {min_count}개")

    # 2. 명시적 반복문을 사용하여 샘플링
    balanced_dfs = []
    for label, group in df.groupby(class_column):
        # 각 클래스 그룹에서 동일하게 min_count만큼 샘플링
        sampled_group = group.sample(min_count, random_state=42)
        balanced_dfs.append(sampled_group)

    # 3. 데이터 합치기 및 셔플
    df_balanced = pd.concat(balanced_dfs)
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f">> 밸런싱 완료! 전체 데이터 개수: {len(df_balanced)}")
    print(f">> 클래스 분포:\n{df_balanced[class_column].value_counts()}")
    
    return df_balanced

---
### tokenizer

In [18]:
# 사전 학습 모델의 토크나이저
class PretrainedHFTokenizer:
    def __init__(self, model_name, config):
        print(f"사전학습된 토크나이저 로드 중: {model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.word2idx = self.tokenizer.get_vocab()
        
        # [수정] 사전학습 모델의 ID를 config에 정확히 동기화
        config.pad_id = self.tokenizer.pad_token_id
        config.unk_id = self.tokenizer.unk_token_id
        config.cls_id = self.tokenizer.cls_token_id
        config.sep_id = self.tokenizer.sep_token_id
        
        # [수정] mask_id도 ID로 가져와서 저장 (없으면 None)
        mask_token = self.tokenizer.mask_token or "[MASK]"
        config.mask_id = self.tokenizer.convert_tokens_to_ids(mask_token)

    def encode(self, text):
        return self.tokenizer.encode(text, add_special_tokens=False)

    def decode(self, ids, skip_special_tokens=False):
        return self.tokenizer.decode(ids, skip_special_tokens=skip_special_tokens)

    def tokenize(self, text):
        return self.tokenizer.tokenize(text)

    def __len__(self):
        return len(self.tokenizer)


In [19]:
# WordPiece
class WordPieceTokenizer:
    def __init__(self, corpus_list, vocab_size, config, model_prefix=None, force_train=False):
        self.vocab_size = vocab_size
        self.model_prefix = model_prefix or f"./wordpiece/wp_v{self.vocab_size}"
        self.json_path = f"{self.model_prefix}.json"
        self.corpus_file = './wordpiece/corpus.txt'

        os.makedirs(os.path.dirname(self.json_path), exist_ok=True)

        # force_train이 True이거나 파일이 없을 때만 학습하도록 논리 수정
        if force_train or not os.path.exists(self.json_path):
            print(f">> [Info] WordPiece 학습을 시작함: {self.json_path}")
            
            # [수정] NoneType 에러 방지를 위한 데이터 검증
            if corpus_list is None:
                raise ValueError("학습을 위한 corpus_list가 None임. 데이터를 확인해야 함.")
            
            # 학습용 말뭉치 파일 생성
            with open(self.corpus_file, 'w', encoding='utf-8') as f:
                for text in corpus_list:
                    if text is not None:  # 요소가 None인 경우 방지
                        f.write(str(text).strip() + "\n")

            # WordPiece 모델 설정
            tokenizer = Tokenizer(models.WordPiece(unk_token=config.unk_token))
            tokenizer.normalizer = normalizers.BertNormalizer() 
            tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
            
            special_tokens = [config.pad_token, config.unk_token, config.cls_token, config.sep_token, config.mask_token]
            trainer = trainers.WordPieceTrainer(vocab_size=self.vocab_size, special_tokens=special_tokens)
            
            # 학습 진행
            tokenizer.train(files=[self.corpus_file], trainer=trainer)
            
            # 모델 전체를 JSON으로 저장
            tokenizer.save(self.json_path)
            print(f">> [Info] WordPiece 학습 및 JSON 저장 완료: {self.json_path}")
        else:
            print(f">> [Info] 기존에 학습된 모델을 로드함: {self.json_path}")

        # 2. 인터페이스 로드 (JSON 파일을 직접 읽음)
        self.tokenizer = PreTrainedTokenizerFast(
            tokenizer_file=self.json_path,
            pad_token=config.pad_token,
            unk_token=config.unk_token,
            cls_token=config.cls_token,
            sep_token=config.sep_token,
            mask_token=config.mask_token
        )
        self.word2idx = self.tokenizer.get_vocab()

    def encode(self, text): return self.tokenizer.encode(text, add_special_tokens=False)
    def tokenize(self, text): return self.tokenizer.tokenize(text)
    def decode(self, ids): return self.tokenizer.decode(ids)
    def __len__(self): return len(self.tokenizer)


In [20]:
# --- SentencePiece 토크나이저 ---
class SPTokenizer:
    def __init__(self, corpus_list, vocab_size, config, model_prefix=None, force_train=False):
        self.vocab_size = vocab_size
        self.corpus_file = './spm/corpus.txt'

        model_tag = f"spm_{config.spm_model_type}_v{self.vocab_size}"
        
        # 2. 경로 설정: 외부에서 지정하지 않으면 태그를 기반으로 기본 경로 생성
        if model_prefix is None:
            self.model_prefix = f'./spm/{model_tag}'
        else:
            self.model_prefix = model_prefix
            
        model_path = f'{self.model_prefix}.model'
        
        # 폴더 생성
        os.makedirs(os.path.dirname(self.corpus_file), exist_ok=True)
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        
        need_train = force_train or not os.path.exists(model_path)

        if not need_train:
            temp_sp = spm.SentencePieceProcessor()
            temp_sp.load(model_path)
            if temp_sp.get_piece_size() != self.vocab_size:
                need_train = True

        if need_train:
            with open(self.corpus_file, 'w', encoding='utf-8') as f:
                for text in corpus_list:
                    f.write(text + "\n")
            
            # mask_id 파라미터 제거 및 user_defined_symbols 리스트화
            spm.SentencePieceTrainer.Train(
                input=self.corpus_file,
                model_prefix=self.model_prefix,
                vocab_size=self.vocab_size,
                character_coverage=0.9995,
                model_type=config.spm_model_type,
                pad_id=config.pad_id,
                unk_id=config.unk_id,
                bos_id=config.cls_id,
                eos_id=config.sep_id,
                pad_piece='[PAD]',
                unk_piece='[UNK]',
                bos_piece='[CLS]',
                eos_piece='[SEP]',
                user_defined_symbols=['[MASK]'] # 리스트로 전달
            )
            
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(model_path)
        self.word2idx = {self.sp.id_to_piece(i): i for i in range(self.sp.get_piece_size())}

    def encode(self, text): return self.sp.encode_as_ids(text)
    def tokenize(self, text): return self.sp.encode_as_pieces(text)
    def decode(self, ids): return self.sp.decode_ids(ids)
    def __len__(self): return self.sp.get_piece_size()


In [21]:
# --- 2.KoNLPy 공통 구조 (속성명 통일) ---
class KoNLPyTokenizer:
    def __init__(self, tagger_type="okt", corpus=None, vocab_size=5000, force_train=False):
        self.vocab_path = f"{tagger_type}_vocab.json"
        self.tagger_type = tagger_type
        self.vocab_size = vocab_size
        self.special_tokens = ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]']
        
        # 1. 태거 초기화
        self._init_tagger(tagger_type)

        os.makedirs(os.path.dirname(self.vocab_path), exist_ok=True)

        # 상황 1: 강제 학습 모드이거나, 저장된 사전 파일이 없는 경우 -> '새로 만들기'
        if force_train or not os.path.exists(self.vocab_path):
            if corpus is not None:
                print(f"사전 생성을 시작합니다. (force_train={force_train})")
                self._build_and_save_vocab(corpus)
            else:
                # 학습할 데이터(corpus)도 없고 파일도 없는 경우의 예외 처리
                print("경고: 저장된 사전 파일이 없고 학습할 데이터(corpus)도 없습니다. 기본 토큰으로 초기화합니다.")
                self.word2idx = {t: i for i, t in enumerate(self.special_tokens)}
                self.idx2word = {i: t for i, t in enumerate(self.special_tokens)}

        # 상황 2: 이미 파일이 존재하고, 새로 학습할 필요가 없는 경우 -> '불러오기'
        else:
            print(f"기존 사전을 로드합니다: {self.vocab_path}")
            self._load_vocab()
    
    def _init_tagger(self, tagger_type):
        if tagger_type == "okt": self.tagger = Okt()
        elif tagger_type == "kkma": self.tagger = Kkma()
        elif tagger_type == "mecab": self.tagger = Mecab()
        else: raise ValueError("지원하지 않는 태거 타입입니다.")

    def _build_and_save_vocab(self, corpus):
        """말뭉치로부터 사전을 만들고 파일로 저장합니다."""
        print(f"새로운 사전을 생성하여 '{self.vocab_path}'에 저장합니다...")
        
        # 토큰화 및 빈도 계산
        all_tokens = [t for text in corpus for t in (self.tagger.morphs(text) or [])]
        counts = Counter(all_tokens)
        
        # 사전 구성
        vocab = self.special_tokens + [w for w, _ in counts.most_common(self.vocab_size - len(self.special_tokens))]
        self.word2idx = {word: i for i, word in enumerate(vocab)}
        self.idx2word = {i: word for word, i in self.word2idx.items()}

        # 파일 저장
        data = {
            'tagger_type': self.tagger_type,
            'word2idx': self.word2idx
        }
        with open(self.vocab_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print("사전 생성 및 저장 완료.")

    def _load_vocab(self):
        """저장된 파일을 읽어와서 객체에 적용합니다."""
        print(f"'{self.vocab_path}'에서 사전을 불러오는 중...")
        with open(self.vocab_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if self.tagger_type != data['tagger_type']:
            print(f"주의: 설정된 태거가 저장된 태거와 다릅니다. 저장된 설정을 따릅니다.")
            self._init_tagger(data['tagger_type'])
            
        self.word2idx = data['word2idx']
        self.idx2word = {int(index): word for word, index in self.word2idx.items()}
        
        print(f"사전 로드 완료. (단어 수: {len(self.word2idx)})")

    def encode(self, text):
        return [self.word2idx.get(t, self.word2idx['[UNK]']) for t in self.tagger.morphs(text)]
    
    def decode(self, ids):
        return " ".join([self.idx2word.get(int(i), '[UNK]') for i in ids])
    
    def tokenize(self, text):
        """텍스트를 인덱싱하지 않고 형태소 단위로만 분리합니다."""
        return self.tagger.morphs(text)

    def __len__(self):
        return len(self.word2idx)

In [22]:
def get_tokenizer(config, corpus=None):
    force_train = getattr(config, 'force_train', False)
    
    # 기본 ID 설정
    if config.tokenizer_type != "pretrained":
        config.pad_id, config.unk_id, config.cls_id, config.sep_id, config.mask_id = 0, 1, 2, 3, 4
        config.mask_token = '[MASK]'

    if config.tokenizer_type == "pretrained":
        tokenizer = PretrainedHFTokenizer(config.pretrained_model_name, config)
    
    elif config.tokenizer_type == "spm":
        tokenizer = SPTokenizer(corpus, config.vocab_size, config, force_train=force_train)
        
    # [추가] WordPiece 분기
    elif config.tokenizer_type == "wordpiece":
        tokenizer = WordPieceTokenizer(corpus, config.vocab_size, config, force_train=force_train)
    
    elif config.tokenizer_type in ["okt", "kkma", "mecab"]:
        tokenizer = KoNLPyTokenizer(config.tokenizer_type, corpus, config.vocab_size, force_train=force_train)
    
    else:
        raise ValueError(f"지원하지 않는 토크나이저 타입: {config.tokenizer_type}")

    return tokenizer

### Train Validation 코드

In [23]:
def get_cosine_with_warmup_lr_lambda(total_steps, warmup_steps, min_lr_ratio=1e-7):
    def lr_lambda(current_step):
        # 1. Linear Warmup 구간
        if current_step < warmup_steps:
            return float(current_step) / float(max(1, warmup_steps))
        
        # 2. Cosine Annealing 구간
        progress = float(current_step - warmup_steps) / float(max(1, total_steps - warmup_steps))
        
        # cosine_decay는 1.0 ~ min_lr_ratio로 변함
        cosine_decay = 0.5 * (1.0 + math.cos(math.pi * progress))
        
        # (1.0 - min_lr_ratio) 범위를 곱해주고 마지막에 min_lr_ratio를 더함
        return min_lr_ratio + (1.0 - min_lr_ratio) * cosine_decay
        
    return lr_lambda


In [24]:
def calculate_metrics_classification(logits, labels):
    """분류 정확도 계산"""
    preds = torch.argmax(logits, dim=-1)
    correct = (preds == labels).sum().item()
    total = labels.size(0)
    return correct, total

In [25]:
# 4. 검증 함수
def validate(model, dataloader, criterion, device, config):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_logits = []
    
    with torch.no_grad():
        for ids, labels in dataloader:
            ids, labels = ids.to(device), labels.to(device)
            
            # [핵심 수정] 패딩 마스크 생성 (0이 아닌 곳만 1)
            mask = (ids != config.pad_id).long()
            
            # [핵심 수정] 모델 호출 시 mask 인자 전달
            # 반환값 구조에 따라 필요시 _ 처리 (예: _, logits = model(...))
            logits = model(ids, mask=mask)
            
            loss = criterion(logits, labels)
            total_loss += loss.item()
            
            preds = torch.argmax(logits, dim=-1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_logits.extend(logits.cpu().numpy())
            
    avg_loss = total_loss / len(dataloader)
    
    # Macro F1: 클래스 불균형이 있을 때 소수 클래스 성능을 잘 반영함
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    # 정확도 계산
    acc = (np.array(all_preds) == np.array(all_labels)).mean()
    
    # 6개의 값을 반환하여 메트릭 확인과 시각화를 동시에 지원
    return avg_loss, acc, f1, all_labels, all_preds, all_logits


In [26]:
# 3. Epoch 학습 함수
def train_one_epoch(model, dataloader, criterion, optimizer, scheduler, device, config):
    model.train()
    total_loss, total_correct, total_total = 0, 0, 0

    for ids, labels in dataloader:
        ids, labels = ids.to(device), labels.to(device)
        
        # [추가] 패딩 마스크 생성: 실제 토큰은 1, 패딩은 0
        # (Accuracy:높음 - ids != pad_id 로직)
        mask = (ids != config.pad_id).long()
        
        optimizer.zero_grad()
        
        # [수정] 모델 호출 시 mask 인자 반드시 전달
        logits = model(ids, mask=mask) 
        
        loss = criterion(logits, labels)
        loss.backward()
        
        # 그래디언트 클리핑: max_norm=1.0 (Accuracy:높음 - BERT 표준)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # 스케줄러 업데이트 (배치 단위)
        scheduler.step()
        
        # 통계 계산
        total_loss += loss.item()
        
        # 정확도 지표 계산 (정답 수 c, 전체 샘플 수 t)
        c, t = calculate_metrics_classification(logits, labels)
        total_correct += c
        total_total += t
        
    avg_loss = total_loss / len(dataloader)
    avg_acc = total_correct / total_total if total_total > 0 else 0
    
    return avg_loss, avg_acc

In [ ]:
def train(config, model, train_loader, val_loader):
    model_tag = model_tagging(config)
    config.model_path = f"./models/best_bert_f1_{model_tag}.pth"
    os.makedirs("./models", exist_ok=True)

    optimizer = optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
    TOTAL_STEPS = len(train_loader) * config.epochs
    scheduler = lr_scheduler.LambdaLR(
        optimizer, lr_lambda=get_cosine_with_warmup_lr_lambda(TOTAL_STEPS, int(TOTAL_STEPS * 0.05))
    )
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    early_stop_counter, best_f1 = 0, 0.0
    best_val_loss = float('inf')
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(config.epochs):
        # 모든 지표를 계산하여 반환
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, scheduler, device, config)
        val_loss, val_acc, val_f1, all_labels, all_preds, all_logits = validate(model, val_loader, criterion, device, config)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch+1}/{config.epochs}")
        print(f"  [Train] Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
        print(f"  [Val]   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}")

        improved_f1 = val_f1 > best_f1
        improved_loss = val_loss < best_val_loss

        if improved_f1 or improved_loss:
            early_stop_counter = 0
            
            if improved_f1 or improved_loss:
                if improved_f1:
                    best_f1 = val_f1
                    print(f"  >> Best F1 Updated! ({val_f1:.4f})")

                if improved_loss:
                    best_val_loss = val_loss
                    print(f"  >> Best Loss Updated! ({val_loss:.4f})")

                early_stop_counter = 0
                save_checkpoint(model, optimizer, config, epoch, val_f1, val_loss, config.model_path)
                print(f"  >> Best model Updated! ({val_f1:.4f}, {val_loss:.4f})")
        else:
            early_stop_counter += 1
            print(f"  >> EarlyStopping counter: {early_stop_counter} out of {config.patience}")

        if early_stop_counter >= config.patience:
            print(f"성능 개선이 없어 {epoch+1} 에포크에서 학습을 조기 종료합니다.")
            break

        print("-" * 50)

    print("학습이 완료되었습니다. 최종 지표 분석을 시작합니다.")

    if os.path.exists(config.model_path):
        checkpoint = torch.load(config.model_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f">> 최고 성능 모델 복원 완료 (Epoch {checkpoint['epoch']+1})")

    # 검증 데이터셋으로 최종 결과 추출
    val_acc, val_acc, val_f1, all_labels, all_preds, all_logits = validate(
        model, val_loader, criterion, device, config
    )
    # print(f"val_acc:{val_acc}, val_acc:{val_acc}, val_f1:{val_f1}")
    # 오타로 수정
    print(f"val_acc:{val_acc:.4f}, val_loss: {val_loss:.4f}, val_f1:{val_f1:.4f}")

    # 최종 시각화 저장 (딱 한 번, 최고의 성능 기준으로)
    results_dir = f"./results/{model_tag}_d{config.d_model}_f{config.ff_dim}_l{config.num_layers}"
    os.makedirs(results_dir, exist_ok=True)

    final_cm_path = os.path.join(results_dir, "final_best_confusion_matrix.png")

    # 혼동 행렬 및 학습 곡선 저장
    plot_and_save_confusion_matrix(all_labels, all_preds, class_names, final_cm_path)
    save_training_plots(history, model_tag, all_labels, all_logits)
    print(f">> 모든 최종 결과물이 ./results/{model_tag} 에 저장되었습니다.")
    
    return history

In [28]:
def test_and_predict(config):
    """
    저장된 최고 성능 모델을 로드하여 테스트 데이터에 대해 추론하고 결과를 저장합니다.
    """
    # 1. 모델 태깅 및 경로 설정
    model_tag = model_tagging(config)

    if not os.path.exists(config.model_path):
        raise FileNotFoundError(f"모델 파일을 찾을 수 없습니다: {config.model_path}")

    # 2. 체크포인트 로드 및 Config 복원
    checkpoint = torch.load(config.model_path , map_location=device)
    config_dict = checkpoint.get('config', {})
    config.__dict__.update(config_dict)  # 학습 당시 하이퍼파라미터 복원
    print(f"모델 가중치 로드 완료 (Epoch: {checkpoint.get('epoch', 'N/A')}, F1: {checkpoint.get('val_f1', 0.0):.4f})")

    # 3. 모델 초기화 및 Eval 모드 설정
    backbone = StandardBertModel(config)
    model = BertForSequenceClassification(backbone, config, num_labels=5).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    print(f"모델 가중치 로드 완료 (Epoch: {checkpoint.get('epoch', 'N/A')}, F1: {checkpoint.get('val_f1', 'N/A'):.4f})")

    # 4. 토크나이저 및 데이터 준비
    config.force_train = False
    tokenizer = get_tokenizer(config)

    with open("./dataset/test.json", "r", encoding="utf-8") as f:
        test_json = json.load(f)

    test_ids = list(test_json.keys())
    raw_texts = [test_json[tid]['text'] for tid in test_ids]
    test_texts = [clean_text(text) for text in raw_texts]

    test_dataset = TestDataset(test_texts, tokenizer, config)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # 5. 추론 수행
    all_preds, all_probs = [], []
    print(f"추론 시작... (테스트 데이터: {len(test_texts)}개)")

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting"):
            # 데이터를 GPU/CPU로 이동
            ids = batch.to(device)
            
            # 패딩 마스크 생성 (BERT 필수 인자)
            mask = (ids != config.pad_id).long()
            
            # 모델 포워딩
            logits = model(ids, mask=mask)
            
            # 확률 변환 및 클래스 결정
            probs = F.softmax(logits, dim=-1)
            max_prob, pred = torch.max(probs, dim=-1)
            
            all_probs.extend(max_prob.cpu().numpy())
            all_preds.extend(pred.cpu().numpy())

    # 6. 결과 정리
    results_list = []
    for tid, text, p, prob in zip(test_ids, raw_texts, all_preds, all_probs):
        submission_label = int(p)
        text_label = class_map[submission_label]

        cleaned_text = clean_text(text)
        tokens = tokenizer.tokenize(cleaned_text)
        tokenized_str = " ".join(tokens)
        
        results_list.append({
            "idx": tid,
            "class": submission_label,
            "text_label": text_label,
            "confidence": round(float(prob), 4),
            "tokenized_text": tokenized_str,
            "conversation": text,
            "cleaned_text": cleaned_text
        })

    df_total = pd.DataFrame(results_list)

    # 7. 파일 저장
    output_dir = f"./dataset/{model_tag}_d{config.d_model}_f{config.ff_dim}_l{config.num_layers}"
    os.makedirs(output_dir, exist_ok=True)
    
    submission_path = os.path.join(output_dir, f"submission_{model_tag}_{config.vocab_size}.csv")
    result_detail_path = os.path.join(output_dir, f"test_result_{model_tag}_{config.vocab_size}.csv")

    df_total[["idx", "class"]].to_csv(submission_path, index=False)
    df_total.to_csv(result_detail_path, index=False, encoding="utf-8-sig")

    print(f"최종 결과 저장 완료: {output_dir}")
    return df_total

### 데이터 불러오기

In [29]:
# 1. 데이터 로드 및 균형 맞추기
df = pd.read_csv("./dataset/train_v2.csv").dropna(subset=["conversation", "class"])
df = balance_dataset(df, class_column='class')

df["label"] = df["class"].map(label_dict)
df["conversation"] = df["conversation"].apply(clean_text)

>> [Data Balancing] 최소 데이터 클래스 개수 기준: 890개
>> 밸런싱 완료! 전체 데이터 개수: 4450
>> 클래스 분포:
class
갈취 대화          890
협박 대화          890
일반 대화          890
기타 괴롭힘 대화      890
직장 내 괴롭힘 대화    890
Name: count, dtype: int64


### Koelectra base v3

In [30]:
config.tokenizer_type = 'pretrained' 
config.pretrained_model_name = 'monologg/koelectra-base-v3-discriminator'
config.num_layers = 4
config.d_model = 256
config.ff_dim = 1024

In [31]:
tokenizer = get_tokenizer(config, df["conversation"].tolist())
config.update_from_tokenizer(tokenizer) # Vocab Size 동기화

사전학습된 토크나이저 로드 중: monologg/koelectra-base-v3-discriminator
[Config Finalized] Vocab: 35000
 >> IDs - PAD:0, UNK:1, CLS:2, SEP:3, MASK:4


In [32]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

train_loader = DataLoader(
    ClassificationDataset(train_df["conversation"].tolist(), train_df["label"].tolist(), tokenizer, config, is_train=True),
    batch_size=config.batch_size, shuffle=True
)
val_loader = DataLoader(
    ClassificationDataset(val_df["conversation"].tolist(), val_df["label"].tolist(), tokenizer, config, is_train=False),
    batch_size=config.batch_size
)
# debug(train_loader, tokenizer)

In [33]:
# 3. 모델 초기화
backbone = StandardBertModel(config)
model = BertForSequenceClassification(backbone, config, num_labels=5).to(device)
model.apply(init_weights) # 가중치 안정화

BertForSequenceClassification(
  (bert): StandardBertModel(
    (token_emb): Embedding(35000, 256)
    (pos_emb): Embedding(128, 256)
    (seg_emb): Embedding(2, 256)
    (norm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (layers): ModuleList(
      (0-3): 4 x ModuleDict(
        (mha): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (norm1): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
        (ffn): Sequential(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.3, inplace=False)
          (3): Linear(in_features=1024, out_features=256, bias=True)
          (4): Dropout(p=0.3, inplace=False)
        )
        (norm2): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      )
    )
    (pooler): Sequential(
      (0): Linear(in_features=256, 

In [34]:
%%time
history = train(
   config, model, train_loader, val_loader
)

Epoch 1/100
  [Train] Loss: 1.6100 | Acc: 0.1980
  [Val]   Loss: 1.6093 | Acc: 0.2000 | F1: 0.0667
  >> Best F1 Updated! (0.0667)
  >> Best Loss Updated! (1.6093)
  >> Best model Updated! (0.0667, 1.6093)
--------------------------------------------------
Epoch 2/100
  [Train] Loss: 1.6094 | Acc: 0.2084
  [Val]   Loss: 1.6081 | Acc: 0.2000 | F1: 0.0667
  >> Best Loss Updated! (1.6081)
  >> Best model Updated! (0.0667, 1.6081)
--------------------------------------------------
Epoch 3/100
  [Train] Loss: 1.6054 | Acc: 0.2284
  [Val]   Loss: 1.5731 | Acc: 0.2730 | F1: 0.1846
  >> Best F1 Updated! (0.1846)
  >> Best Loss Updated! (1.5731)
  >> Best model Updated! (0.1846, 1.5731)
--------------------------------------------------
Epoch 4/100
  [Train] Loss: 1.4133 | Acc: 0.4320
  [Val]   Loss: 1.2820 | Acc: 0.5787 | F1: 0.5139
  >> Best F1 Updated! (0.5139)
  >> Best Loss Updated! (1.2820)
  >> Best model Updated! (0.5139, 1.2820)
--------------------------------------------------
Epoch 5

In [35]:
test_and_predict(config)

모델 가중치 로드 완료 (Epoch: 12, F1: 0.8221)
모델 가중치 로드 완료 (Epoch: 12, F1: 0.8221)
사전학습된 토크나이저 로드 중: monologg/koelectra-base-v3-discriminator
추론 시작... (테스트 데이터: 500개)


Predicting: 100%|██████████| 16/16 [00:00<00:00, 49.08it/s]


최종 결과 저장 완료: ./dataset/monologg_koelectra-base-v3-discriminator_d256_f1024_l4


,idx,class,text_label,confidence,tokenized_text,conversation,cleaned_text
0,t_000,1,갈취 대화,0.9128,아가씨 담배 ##한 ##갑 ##주 ##소 네 원 ##입니다 어 네 지갑 ##어 ##...,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,아가씨 담배한갑주소 네 원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나보네 그...
1,t_001,2,직장 내 괴롭힘 대화,0.9197,우리 ##팀 ##에 ##서 다른 ##팀 ##으로 갈 사람 없 ##나 그럼 영지 ##...,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 달...
2,t_002,2,직장 내 괴롭힘 대화,0.8995,너 오늘 그게 뭐 ##야 네 제 ##가 뭘 잘못 ##했 ##나 ##요 제대로 좀 하...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
3,t_003,3,기타 괴롭힘 대화,0.8941,이거 들어 ##바 와 이 노래 진짜 좋 ##다 그치 요즘 이 것 ##만 들어 진짜 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,3,기타 괴롭힘 대화,0.9081,아무튼 앞 ##으로 니 ##가 내 와이파이 ##야 응 와이파이 온 켰 ##어 반말 ...,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...
...,...,...,...,...,...,...,...
495,t_495,2,직장 내 괴롭힘 대화,0.9214,미나 ##씨 휴가 결제 올리 ##기 전 ##에 저 ##랑 상의 ##하라 ##고 말 ...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요 네 합니다 보고서를 ...
496,t_496,1,갈취 대화,0.4080,교수 ##님 제 논문 ##에 제 이름 ##이 없 ##나 ##요 아 무슨 논문 ##말...,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,교수님 제 논문에 제 이름이 없나요 아 무슨 논문말이야 지난 번 냈던 논문이요 그거...
497,t_497,1,갈취 대화,0.9090,야 너 네 저 ##요 그래 너 왜 ##요 돈 ##좀 줘 ##봐 돈 없 ##어요 돈 ...,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,야 너 네 저요 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이니 진...
498,t_498,2,직장 내 괴롭힘 대화,0.8755,야 너 빨리 안 뛰어 ##와 너 이 환자 제대로 봤 ##어 안 봤 ##어 어제 저녁...,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,야 너 빨리 안 뛰어와 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다가...


### klue/bert-base

In [36]:
config.tokenizer_type = 'pretrained' 
config.pretrained_model_name = "klue/bert-base"
config.num_layers = 4
config.d_model = 256
config.ff_dim = 1024

In [37]:
tokenizer = get_tokenizer(config, df["conversation"].tolist())
config.update_from_tokenizer(tokenizer) # Vocab Size 동기화

사전학습된 토크나이저 로드 중: klue/bert-base
[Config Finalized] Vocab: 32000
 >> IDs - PAD:0, UNK:1, CLS:2, SEP:3, MASK:4


In [38]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

train_loader = DataLoader(
    ClassificationDataset(train_df["conversation"].tolist(), train_df["label"].tolist(), tokenizer, config, is_train=True),
    batch_size=config.batch_size, shuffle=True
)
val_loader = DataLoader(
    ClassificationDataset(val_df["conversation"].tolist(), val_df["label"].tolist(), tokenizer, config, is_train=False),
    batch_size=config.batch_size
)
# debug(train_loader, tokenizer)

In [39]:
# 3. 모델 초기화
backbone = StandardBertModel(config)
model = BertForSequenceClassification(backbone, config, num_labels=5).to(device)
model.apply(init_weights) # 가중치 안정화

BertForSequenceClassification(
  (bert): StandardBertModel(
    (token_emb): Embedding(32000, 256)
    (pos_emb): Embedding(128, 256)
    (seg_emb): Embedding(2, 256)
    (norm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (layers): ModuleList(
      (0-3): 4 x ModuleDict(
        (mha): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (norm1): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
        (ffn): Sequential(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.3, inplace=False)
          (3): Linear(in_features=1024, out_features=256, bias=True)
          (4): Dropout(p=0.3, inplace=False)
        )
        (norm2): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      )
    )
    (pooler): Sequential(
      (0): Linear(in_features=256, 

In [40]:
%%time
history = train(
   config, model, train_loader, val_loader
)

Epoch 1/100
  [Train] Loss: 1.6101 | Acc: 0.1994
  [Val]   Loss: 1.6093 | Acc: 0.2000 | F1: 0.0667
  >> Best F1 Updated! (0.0667)
  >> Best Loss Updated! (1.6093)
  >> Best model Updated! (0.0667, 1.6093)
--------------------------------------------------
Epoch 2/100
  [Train] Loss: 1.6094 | Acc: 0.2003
  [Val]   Loss: 1.6087 | Acc: 0.2000 | F1: 0.0667
  >> Best Loss Updated! (1.6087)
  >> Best model Updated! (0.0667, 1.6087)
--------------------------------------------------
Epoch 3/100
  [Train] Loss: 1.6076 | Acc: 0.2295
  [Val]   Loss: 1.5909 | Acc: 0.3517 | F1: 0.2109
  >> Best F1 Updated! (0.2109)
  >> Best Loss Updated! (1.5909)
  >> Best model Updated! (0.2109, 1.5909)
--------------------------------------------------
Epoch 4/100
  [Train] Loss: 1.4035 | Acc: 0.4857
  [Val]   Loss: 1.2067 | Acc: 0.6169 | F1: 0.5533
  >> Best F1 Updated! (0.5533)
  >> Best Loss Updated! (1.2067)
  >> Best model Updated! (0.5533, 1.2067)
--------------------------------------------------
Epoch 5

In [41]:
test_and_predict(config)

모델 가중치 로드 완료 (Epoch: 13, F1: 0.8277)
모델 가중치 로드 완료 (Epoch: 13, F1: 0.8277)
사전학습된 토크나이저 로드 중: klue/bert-base
추론 시작... (테스트 데이터: 500개)


Predicting: 100%|██████████| 16/16 [00:00<00:00, 47.53it/s]


최종 결과 저장 완료: ./dataset/klue_bert-base_d256_f1024_l4


,idx,class,text_label,confidence,tokenized_text,conversation,cleaned_text
0,t_000,1,갈취 대화,0.9409,아가씨 담배 ##한 ##갑 ##주 ##소 네 원 ##입니다 어 네 지갑 ##어 ##...,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,아가씨 담배한갑주소 네 원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나보네 그...
1,t_001,2,직장 내 괴롭힘 대화,0.9411,우리 ##팀 ##에서 다른 ##팀 ##으로 갈 사람 없 ##나 그럼 영지 ##씨 #...,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 달...
2,t_002,2,직장 내 괴롭힘 대화,0.9412,너 오늘 그게 뭐 ##야 네 제 ##가 뭘 잘못 ##했 ##나 ##요 제대로 좀 하...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
3,t_003,4,일반 대화,0.8586,이거 들어 ##바 와 이 노래 진짜 좋 ##다 그치 요즘 이 것 ##만 들어 진짜 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,2,직장 내 괴롭힘 대화,0.6463,아무튼 앞 ##으로 니 ##가 내 와이파이 ##야 응 와이파이 온 켰 ##어 반말 ...,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...
...,...,...,...,...,...,...,...
495,t_495,2,직장 내 괴롭힘 대화,0.9401,미나 ##씨 휴가 결제 올리 ##기 전 ##에 저 ##랑 상의 ##하라 ##고 말 ...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요 네 합니다 보고서를 ...
496,t_496,2,직장 내 괴롭힘 대화,0.8516,교수 ##님 제 논문 ##에 제 이름 ##이 없 ##나 ##요 아 무슨 논문 ##말...,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,교수님 제 논문에 제 이름이 없나요 아 무슨 논문말이야 지난 번 냈던 논문이요 그거...
497,t_497,1,갈취 대화,0.9275,야 너 네 저 ##요 그래 너 왜 ##요 돈 ##좀 줘 ##봐 돈 없 ##어요 돈 ...,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,야 너 네 저요 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이니 진...
498,t_498,2,직장 내 괴롭힘 대화,0.8289,야 너 빨리 안 뛰어 ##와 너 이 환자 제대로 봤 ##어 안 봤 ##어 어제 저녁...,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,야 너 빨리 안 뛰어와 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다가...


### klue/bert-base - d_modelX2

In [42]:
config.tokenizer_type = 'pretrained' 
config.pretrained_model_name = "klue/bert-base"
config.num_layers = 4
config.d_model = 512
config.ff_dim = 1024

In [43]:
tokenizer = get_tokenizer(config, df["conversation"].tolist())
config.update_from_tokenizer(tokenizer) # Vocab Size 동기화

사전학습된 토크나이저 로드 중: klue/bert-base
[Config Finalized] Vocab: 32000
 >> IDs - PAD:0, UNK:1, CLS:2, SEP:3, MASK:4


In [44]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

train_loader = DataLoader(
    ClassificationDataset(train_df["conversation"].tolist(), train_df["label"].tolist(), tokenizer, config, is_train=True),
    batch_size=config.batch_size, shuffle=True
)
val_loader = DataLoader(
    ClassificationDataset(val_df["conversation"].tolist(), val_df["label"].tolist(), tokenizer, config, is_train=False),
    batch_size=config.batch_size
)
# debug(train_loader, tokenizer)

In [45]:
# 3. 모델 초기화
backbone = StandardBertModel(config)
model = BertForSequenceClassification(backbone, config, num_labels=5).to(device)
model.apply(init_weights) # 가중치 안정화

BertForSequenceClassification(
  (bert): StandardBertModel(
    (token_emb): Embedding(32000, 512)
    (pos_emb): Embedding(128, 512)
    (seg_emb): Embedding(2, 512)
    (norm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (layers): ModuleList(
      (0-3): 4 x ModuleDict(
        (mha): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
        (ffn): Sequential(
          (0): Linear(in_features=512, out_features=1024, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.3, inplace=False)
          (3): Linear(in_features=1024, out_features=512, bias=True)
          (4): Dropout(p=0.3, inplace=False)
        )
        (norm2): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      )
    )
    (pooler): Sequential(
      (0): Linear(in_features=512, 

In [46]:
%%time
history = train(
   config, model, train_loader, val_loader
)

Epoch 1/100
  [Train] Loss: 1.6105 | Acc: 0.1986
  [Val]   Loss: 1.6081 | Acc: 0.2146 | F1: 0.0943
  >> Best F1 Updated! (0.0943)
  >> Best Loss Updated! (1.6081)
  >> Best model Updated! (0.0943, 1.6081)
--------------------------------------------------
Epoch 2/100
  [Train] Loss: 1.6092 | Acc: 0.2096
  [Val]   Loss: 1.5968 | Acc: 0.2247 | F1: 0.1124
  >> Best F1 Updated! (0.1124)
  >> Best Loss Updated! (1.5968)
  >> Best model Updated! (0.1124, 1.5968)
--------------------------------------------------
Epoch 3/100
  [Train] Loss: 1.4423 | Acc: 0.4354
  [Val]   Loss: 1.1774 | Acc: 0.6011 | F1: 0.5586
  >> Best F1 Updated! (0.5586)
  >> Best Loss Updated! (1.1774)
  >> Best model Updated! (0.5586, 1.1774)
--------------------------------------------------
Epoch 4/100
  [Train] Loss: 1.0630 | Acc: 0.6772
  [Val]   Loss: 0.9525 | Acc: 0.7449 | F1: 0.7306
  >> Best F1 Updated! (0.7306)
  >> Best Loss Updated! (0.9525)
  >> Best model Updated! (0.7306, 0.9525)
---------------------------

In [47]:
test_and_predict(config)

모델 가중치 로드 완료 (Epoch: 7, F1: 0.8327)
모델 가중치 로드 완료 (Epoch: 7, F1: 0.8327)
사전학습된 토크나이저 로드 중: klue/bert-base
추론 시작... (테스트 데이터: 500개)


Predicting: 100%|██████████| 16/16 [00:00<00:00, 33.71it/s]


최종 결과 저장 완료: ./dataset/klue_bert-base_d512_f1024_l4


,idx,class,text_label,confidence,tokenized_text,conversation,cleaned_text
0,t_000,1,갈취 대화,0.9021,아가씨 담배 ##한 ##갑 ##주 ##소 네 원 ##입니다 어 네 지갑 ##어 ##...,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,아가씨 담배한갑주소 네 원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나보네 그...
1,t_001,2,직장 내 괴롭힘 대화,0.9300,우리 ##팀 ##에서 다른 ##팀 ##으로 갈 사람 없 ##나 그럼 영지 ##씨 #...,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 달...
2,t_002,2,직장 내 괴롭힘 대화,0.8324,너 오늘 그게 뭐 ##야 네 제 ##가 뭘 잘못 ##했 ##나 ##요 제대로 좀 하...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
3,t_003,4,일반 대화,0.8820,이거 들어 ##바 와 이 노래 진짜 좋 ##다 그치 요즘 이 것 ##만 들어 진짜 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,3,기타 괴롭힘 대화,0.9348,아무튼 앞 ##으로 니 ##가 내 와이파이 ##야 응 와이파이 온 켰 ##어 반말 ...,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...
...,...,...,...,...,...,...,...
495,t_495,2,직장 내 괴롭힘 대화,0.9260,미나 ##씨 휴가 결제 올리 ##기 전 ##에 저 ##랑 상의 ##하라 ##고 말 ...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요 네 합니다 보고서를 ...
496,t_496,3,기타 괴롭힘 대화,0.9289,교수 ##님 제 논문 ##에 제 이름 ##이 없 ##나 ##요 아 무슨 논문 ##말...,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,교수님 제 논문에 제 이름이 없나요 아 무슨 논문말이야 지난 번 냈던 논문이요 그거...
497,t_497,1,갈취 대화,0.9241,야 너 네 저 ##요 그래 너 왜 ##요 돈 ##좀 줘 ##봐 돈 없 ##어요 돈 ...,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,야 너 네 저요 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이니 진...
498,t_498,2,직장 내 괴롭힘 대화,0.8483,야 너 빨리 안 뛰어 ##와 너 이 환자 제대로 봤 ##어 안 봤 ##어 어제 저녁...,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,야 너 빨리 안 뛰어와 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다가...


### klue/bert-base - ff_dim*2

In [48]:
config.tokenizer_type = 'pretrained' 
config.pretrained_model_name = "klue/bert-base"
config.num_layers = 4
config.d_model = 256
config.ff_dim = 2048

In [49]:
tokenizer = get_tokenizer(config, df["conversation"].tolist())
config.update_from_tokenizer(tokenizer) # Vocab Size 동기화

사전학습된 토크나이저 로드 중: klue/bert-base
[Config Finalized] Vocab: 32000
 >> IDs - PAD:0, UNK:1, CLS:2, SEP:3, MASK:4


In [50]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

train_loader = DataLoader(
    ClassificationDataset(train_df["conversation"].tolist(), train_df["label"].tolist(), tokenizer, config, is_train=True),
    batch_size=config.batch_size, shuffle=True
)
val_loader = DataLoader(
    ClassificationDataset(val_df["conversation"].tolist(), val_df["label"].tolist(), tokenizer, config, is_train=False),
    batch_size=config.batch_size
)
# debug(train_loader, tokenizer)

In [51]:
# 3. 모델 초기화
backbone = StandardBertModel(config)
model = BertForSequenceClassification(backbone, config, num_labels=5).to(device)
model.apply(init_weights) # 가중치 안정화

BertForSequenceClassification(
  (bert): StandardBertModel(
    (token_emb): Embedding(32000, 256)
    (pos_emb): Embedding(128, 256)
    (seg_emb): Embedding(2, 256)
    (norm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (layers): ModuleList(
      (0-3): 4 x ModuleDict(
        (mha): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (norm1): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
        (ffn): Sequential(
          (0): Linear(in_features=256, out_features=2048, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.3, inplace=False)
          (3): Linear(in_features=2048, out_features=256, bias=True)
          (4): Dropout(p=0.3, inplace=False)
        )
        (norm2): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      )
    )
    (pooler): Sequential(
      (0): Linear(in_features=256, 

In [52]:
%%time
history = train(
   config, model, train_loader, val_loader
)

Epoch 1/100
  [Train] Loss: 1.6103 | Acc: 0.1890
  [Val]   Loss: 1.6096 | Acc: 0.2000 | F1: 0.0667
  >> Best F1 Updated! (0.0667)
  >> Best Loss Updated! (1.6096)
  >> Best model Updated! (0.0667, 1.6096)
--------------------------------------------------
Epoch 2/100
  [Train] Loss: 1.6094 | Acc: 0.1997
  [Val]   Loss: 1.6084 | Acc: 0.2022 | F1: 0.0716
  >> Best F1 Updated! (0.0716)
  >> Best Loss Updated! (1.6084)
  >> Best model Updated! (0.0716, 1.6084)
--------------------------------------------------
Epoch 3/100
  [Train] Loss: 1.6079 | Acc: 0.2115
  [Val]   Loss: 1.5983 | Acc: 0.2191 | F1: 0.1042
  >> Best F1 Updated! (0.1042)
  >> Best Loss Updated! (1.5983)
  >> Best model Updated! (0.1042, 1.5983)
--------------------------------------------------
Epoch 4/100
  [Train] Loss: 1.4268 | Acc: 0.4500
  [Val]   Loss: 1.2222 | Acc: 0.5787 | F1: 0.5161
  >> Best F1 Updated! (0.5161)
  >> Best Loss Updated! (1.2222)
  >> Best model Updated! (0.5161, 1.2222)
---------------------------

In [53]:
test_and_predict(config)

모델 가중치 로드 완료 (Epoch: 9, F1: 0.8266)
모델 가중치 로드 완료 (Epoch: 9, F1: 0.8266)
사전학습된 토크나이저 로드 중: klue/bert-base
추론 시작... (테스트 데이터: 500개)


Predicting: 100%|██████████| 16/16 [00:00<00:00, 36.60it/s]


최종 결과 저장 완료: ./dataset/klue_bert-base_d256_f2048_l4


,idx,class,text_label,confidence,tokenized_text,conversation,cleaned_text
0,t_000,1,갈취 대화,0.8020,아가씨 담배 ##한 ##갑 ##주 ##소 네 원 ##입니다 어 네 지갑 ##어 ##...,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,아가씨 담배한갑주소 네 원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나보네 그...
1,t_001,2,직장 내 괴롭힘 대화,0.9159,우리 ##팀 ##에서 다른 ##팀 ##으로 갈 사람 없 ##나 그럼 영지 ##씨 #...,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 달...
2,t_002,2,직장 내 괴롭힘 대화,0.9139,너 오늘 그게 뭐 ##야 네 제 ##가 뭘 잘못 ##했 ##나 ##요 제대로 좀 하...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
3,t_003,4,일반 대화,0.8760,이거 들어 ##바 와 이 노래 진짜 좋 ##다 그치 요즘 이 것 ##만 들어 진짜 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,3,기타 괴롭힘 대화,0.8683,아무튼 앞 ##으로 니 ##가 내 와이파이 ##야 응 와이파이 온 켰 ##어 반말 ...,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...
...,...,...,...,...,...,...,...
495,t_495,2,직장 내 괴롭힘 대화,0.9102,미나 ##씨 휴가 결제 올리 ##기 전 ##에 저 ##랑 상의 ##하라 ##고 말 ...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요 네 합니다 보고서를 ...
496,t_496,0,협박 대화,0.6954,교수 ##님 제 논문 ##에 제 이름 ##이 없 ##나 ##요 아 무슨 논문 ##말...,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,교수님 제 논문에 제 이름이 없나요 아 무슨 논문말이야 지난 번 냈던 논문이요 그거...
497,t_497,1,갈취 대화,0.8662,야 너 네 저 ##요 그래 너 왜 ##요 돈 ##좀 줘 ##봐 돈 없 ##어요 돈 ...,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,야 너 네 저요 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이니 진...
498,t_498,2,직장 내 괴롭힘 대화,0.6819,야 너 빨리 안 뛰어 ##와 너 이 환자 제대로 봤 ##어 안 봤 ##어 어제 저녁...,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,야 너 빨리 안 뛰어와 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다가...


### klue/bert-base num_layersX2

In [54]:
config.tokenizer_type = 'pretrained' 
config.pretrained_model_name = "klue/bert-base"
config.num_layers = 8
config.d_model = 256
config.ff_dim = 1024

In [55]:
tokenizer = get_tokenizer(config, df["conversation"].tolist())
config.update_from_tokenizer(tokenizer) # Vocab Size 동기화

사전학습된 토크나이저 로드 중: klue/bert-base
[Config Finalized] Vocab: 32000
 >> IDs - PAD:0, UNK:1, CLS:2, SEP:3, MASK:4


In [56]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

train_loader = DataLoader(
    ClassificationDataset(train_df["conversation"].tolist(), train_df["label"].tolist(), tokenizer, config, is_train=True),
    batch_size=config.batch_size, shuffle=True
)
val_loader = DataLoader(
    ClassificationDataset(val_df["conversation"].tolist(), val_df["label"].tolist(), tokenizer, config, is_train=False),
    batch_size=config.batch_size
)
# debug(train_loader, tokenizer)

In [57]:
# 3. 모델 초기화
backbone = StandardBertModel(config)
model = BertForSequenceClassification(backbone, config, num_labels=5).to(device)
model.apply(init_weights) # 가중치 안정화

BertForSequenceClassification(
  (bert): StandardBertModel(
    (token_emb): Embedding(32000, 256)
    (pos_emb): Embedding(128, 256)
    (seg_emb): Embedding(2, 256)
    (norm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (layers): ModuleList(
      (0-7): 8 x ModuleDict(
        (mha): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (norm1): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
        (ffn): Sequential(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.3, inplace=False)
          (3): Linear(in_features=1024, out_features=256, bias=True)
          (4): Dropout(p=0.3, inplace=False)
        )
        (norm2): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      )
    )
    (pooler): Sequential(
      (0): Linear(in_features=256, 

In [58]:
%%time
history = train(
   config, model, train_loader, val_loader
)

Epoch 1/100
  [Train] Loss: 1.6102 | Acc: 0.1994
  [Val]   Loss: 1.6093 | Acc: 0.2000 | F1: 0.0667
  >> Best F1 Updated! (0.0667)
  >> Best Loss Updated! (1.6093)
  >> Best model Updated! (0.0667, 1.6093)
--------------------------------------------------
Epoch 2/100
  [Train] Loss: 1.6098 | Acc: 0.2008
  [Val]   Loss: 1.6075 | Acc: 0.2146 | F1: 0.1070
  >> Best F1 Updated! (0.1070)
  >> Best Loss Updated! (1.6075)
  >> Best model Updated! (0.1070, 1.6075)
--------------------------------------------------
Epoch 3/100
  [Train] Loss: 1.5905 | Acc: 0.2559
  [Val]   Loss: 1.4509 | Acc: 0.4112 | F1: 0.3076
  >> Best F1 Updated! (0.3076)
  >> Best Loss Updated! (1.4509)
  >> Best model Updated! (0.3076, 1.4509)
--------------------------------------------------
Epoch 4/100
  [Train] Loss: 1.3683 | Acc: 0.4160
  [Val]   Loss: 1.3076 | Acc: 0.5281 | F1: 0.4728
  >> Best F1 Updated! (0.4728)
  >> Best Loss Updated! (1.3076)
  >> Best model Updated! (0.4728, 1.3076)
---------------------------

In [59]:
test_and_predict(config)

모델 가중치 로드 완료 (Epoch: 9, F1: 0.8298)
모델 가중치 로드 완료 (Epoch: 9, F1: 0.8298)
사전학습된 토크나이저 로드 중: klue/bert-base
추론 시작... (테스트 데이터: 500개)


Predicting: 100%|██████████| 16/16 [00:00<00:00, 43.06it/s]


최종 결과 저장 완료: ./dataset/klue_bert-base_d256_f1024_l8


,idx,class,text_label,confidence,tokenized_text,conversation,cleaned_text
0,t_000,1,갈취 대화,0.7285,아가씨 담배 ##한 ##갑 ##주 ##소 네 원 ##입니다 어 네 지갑 ##어 ##...,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,아가씨 담배한갑주소 네 원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나보네 그...
1,t_001,2,직장 내 괴롭힘 대화,0.8577,우리 ##팀 ##에서 다른 ##팀 ##으로 갈 사람 없 ##나 그럼 영지 ##씨 #...,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 달...
2,t_002,2,직장 내 괴롭힘 대화,0.5399,너 오늘 그게 뭐 ##야 네 제 ##가 뭘 잘못 ##했 ##나 ##요 제대로 좀 하...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
3,t_003,4,일반 대화,0.7051,이거 들어 ##바 와 이 노래 진짜 좋 ##다 그치 요즘 이 것 ##만 들어 진짜 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,3,기타 괴롭힘 대화,0.8351,아무튼 앞 ##으로 니 ##가 내 와이파이 ##야 응 와이파이 온 켰 ##어 반말 ...,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...
...,...,...,...,...,...,...,...
495,t_495,2,직장 내 괴롭힘 대화,0.8543,미나 ##씨 휴가 결제 올리 ##기 전 ##에 저 ##랑 상의 ##하라 ##고 말 ...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요 네 합니다 보고서를 ...
496,t_496,0,협박 대화,0.5493,교수 ##님 제 논문 ##에 제 이름 ##이 없 ##나 ##요 아 무슨 논문 ##말...,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,교수님 제 논문에 제 이름이 없나요 아 무슨 논문말이야 지난 번 냈던 논문이요 그거...
497,t_497,1,갈취 대화,0.8242,야 너 네 저 ##요 그래 너 왜 ##요 돈 ##좀 줘 ##봐 돈 없 ##어요 돈 ...,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,야 너 네 저요 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이니 진...
498,t_498,2,직장 내 괴롭힘 대화,0.5860,야 너 빨리 안 뛰어 ##와 너 이 환자 제대로 봤 ##어 안 봤 ##어 어제 저녁...,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,야 너 빨리 안 뛰어와 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다가...
